___
# 1. Download Dependencies

In [1]:
# %%capture installation
%pip install gdown
# %pip install transformers
# %pip install peft accelerate bitsandbytes triton
%pip install evaluate sacrebleu rouge_score
# !sudo apt install tree

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=e0450eec5cdc87bebdbc5d2a8b185d2e153cace03fab0b7de64957a60d29d14d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This beh

In [2]:
# %%capture download_files
!gdown 1fij-ftlExnP9X7l0Z0EvJHjRLzO1689c # train  https://drive.google.com/file/d/1fij-ftlExnP9X7l0Z0EvJHjRLzO1689c/view?usp=drive_link
!gdown 1D8_p329vuN99_kPak-yZ9NUNKR8CS-hJ # val    https://drive.google.com/file/d/1D8_p329vuN99_kPak-yZ9NUNKR8CS-hJ/view?usp=drive_link
!gdown 1XUNXKCX2KxroPekMEo59St7sx-STQM1g # test   https://drive.google.com/file/d/1XUNXKCX2KxroPekMEo59St7sx-STQM1g/view?usp=drive_link

Downloading...
From: https://drive.google.com/uc?id=1fij-ftlExnP9X7l0Z0EvJHjRLzO1689c
To: /kaggle/working/PHOENIX-2014-T.train.csv
100%|███████████████████████████████████████| 1.74M/1.74M [00:00<00:00, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D8_p329vuN99_kPak-yZ9NUNKR8CS-hJ
To: /kaggle/working/PHOENIX-2014-T.dev.csv
100%|████████████████████████████████████████| 120k/120k [00:00<00:00, 81.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XUNXKCX2KxroPekMEo59St7sx-STQM1g
To: /kaggle/working/PHOENIX-2014-T.test.csv
100%|████████████████████████████████████████| 140k/140k [00:00<00:00, 84.0MB/s]


In [3]:
import gc
import os

import numpy as np
import evaluate
import torch
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    get_scheduler,
)

2025-09-18 06:12:22.226088: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758175942.415309      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758175942.472674      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


___
# 2. Configurations

In [4]:
# MODEL_NAME: str = "facebook/mbart-large-50-many-to-many-mmt"
# SRC_LANG: str = "de_DE"  # German
# TGT_LANG: str = "vi_VN"  # Vietnamese
# MODEL_NAME: str = "Shahm/bart-german"
# MODEL_NAME: str = "Helsinki-NLP/opus-mt-de-de"
# MODEL_NAME: str = "Helsinki-NLP/opus-mt-de-vi"
MODEL_NAME: str = "vinai/bartpho-word"

# Dataset - Optimized for German Gloss → Vietnamese
MAX_INPUT_LENGTH: int = 128  # Increased for longer German gloss sequences
MAX_TARGET_LENGTH: int = 128  # Increased for better Vietnamese sentence generation
BATCH_SIZE: int = 12  # Optimized for T4 GPU memory

# LoRA Configuration - Optimized for translation quality
LORA_R: int = 32  # Higher rank for better capacity
LORA_ALPHA: int = 64  # 2x rank for optimal scaling
LORA_DROPOUT: float = 0.05  # Lower dropout for better learning
LORA_TARGET_MODULES: list[str] = [
    "q_proj",
    "k_proj",
    "v_proj",
    "out_proj",  # All attention layers
    "fc1",
    "fc2",  # Feed-forward layers for better translation
]

# Training Configuration
OUTPUT_DIR = "./gloss2vn_model"  # More descriptive name
GRADIENT_ACCUMULATION_STEPS = 4  # Adjusted for larger batch size
EPOCHS = 30  # More epochs for better convergence
LEARNING_RATE = 5e-4  # Higher learning rate for faster convergence
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01

# Optimization
# USE_GRADIENT_CHECKPOINTING = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"CUDA devices available: {torch.cuda.device_count()}")
device

CUDA devices available: 1


device(type='cuda')

___
# 3. Dataset Loader and Preprocess

In [5]:
dataset = load_dataset(
    "csv",
    data_files={
        "train": "PHOENIX-2014-T.train.csv",
        "val": "PHOENIX-2014-T.dev.csv",
        "test": "PHOENIX-2014-T.test.csv",
    },
    delimiter="|",
)
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['geGloss', 'geSentence', 'viSentence'],
        num_rows: 7063
    })
    val: Dataset({
        features: ['geGloss', 'geSentence', 'viSentence'],
        num_rows: 519
    })
    test: Dataset({
        features: ['geGloss', 'geSentence', 'viSentence'],
        num_rows: 642
    })
})

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.src_lang = SRC_LANG
# tokenizer.tgt_lang = TGT_LANG

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


def preprocess_function(examples):
    # Enhanced preprocessing for German gloss → Vietnamese translation
    inputs = []
    targets = []

    for gloss, sentence in zip(examples["geGloss"], examples["viSentence"]):
        # Clean and normalize German gloss
        clean_gloss = gloss.strip(" .").replace("  ", " ")  # Remove extra spaces
        # Add special prefix to indicate gloss-to-text translation
        # formatted_input = f"German gloss: {clean_gloss}"
        formatted_input = clean_gloss

        # Clean Vietnamese target
        clean_target = sentence.strip(" .").replace("  ", " ")

        inputs.append(formatted_input)
        targets.append(clean_target)

    # Tokenize with improved settings
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding=False,  # Let data collator handle padding
        add_special_tokens=True,
        return_attention_mask=True,
    )

    # Tokenize targets with proper language setting
    labels = tokenizer(
        text_target=targets,
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding=False,
        add_special_tokens=True,
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = dataset.map(
    preprocess_function, batched=True, remove_columns=dataset["train"].column_names, desc="Tokenizing datasets"
)

# Data Collator will dynamically pad the batches
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=MODEL_NAME,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,  # Optimize for tensor cores
    return_tensors="pt"
)

# Create DataLoaders
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=BATCH_SIZE,
    pin_memory=True,
)
eval_dataloader = DataLoader(
    tokenized_datasets["val"],
    collate_fn=data_collator,
    batch_size=BATCH_SIZE * 2,
    pin_memory=True,
)
test_dataloader = DataLoader(
    tokenized_datasets["test"],
    collate_fn=data_collator,
    batch_size=BATCH_SIZE * 2,
    pin_memory=True,
)

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenizing datasets:   0%|          | 0/7063 [00:00<?, ? examples/s]

Tokenizing datasets:   0%|          | 0/519 [00:00<?, ? examples/s]

Tokenizing datasets:   0%|          | 0/642 [00:00<?, ? examples/s]

___
# 4. Model Loader

In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    use_safetensors=True,
)

# Move model to device
model = model.to(device)

# Create optimized LoRA config
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    target_modules=LORA_TARGET_MODULES,
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
)

# Apply LoRA
model = get_peft_model(model, lora_config)

# Ensure LoRA parameters require gradients
for name, param in model.named_parameters():
    if "lora_" in name:
        param.requires_grad_(True)

model.train()
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

trainable params: 17,301,504 || all params: 437,662,720 || trainable%: 3.9532


In [8]:
trainable_params = [p for p in model.parameters() if p.requires_grad]

# Optimized optimizer with better settings
optimizer = torch.optim.AdamW(
    trainable_params,
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    betas=(0.9, 0.98),
    eps=1e-6,
)

# Advanced learning rate scheduler
num_update_steps_per_epoch = len(train_dataloader) // GRADIENT_ACCUMULATION_STEPS
num_training_steps = EPOCHS * num_update_steps_per_epoch
num_warmup_steps = int(num_training_steps * WARMUP_RATIO)

lr_scheduler = get_scheduler(
    name="cosine",
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)
f"Training steps: {num_training_steps}, Warmup: {num_warmup_steps}"

'Training steps: 4410, Warmup: 441'

___
# 5. Metrics Definition

In [9]:
# Load metrics
bleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")
chrf_metric = evaluate.load("chrf")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating", leave=False):
            batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}

            # Enhanced generation for better quality
            generated_tokens = model.generate(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=MAX_TARGET_LENGTH,
                min_length=10,  # Ensure minimum translation length
                num_beams=6,  # More beams for better search
                length_penalty=1.2,  # Encourage longer, more complete translations
                early_stopping=True,
                do_sample=False,
                repetition_penalty=1.1,  # Reduce repetition
                no_repeat_ngram_size=3,  # Prevent 3-gram repetition
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                # forced_bos_token_id=tokenizer.lang_code_to_id[TGT_LANG], # BART mnt-mnt
            )

            labels = batch["labels"]
            labels = np.where(
                labels.cpu() != -100, labels.cpu(), tokenizer.pad_token_id
            )

            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            post_preds, post_labels = postprocess_text(decoded_preds, decoded_labels)
            all_preds.extend(post_preds)
            all_labels.extend(post_labels)

    # Compute metrics
    bleu = bleu_metric.compute(predictions=all_preds, references=all_labels)
    rouge = rouge_metric.compute(predictions=all_preds, references=all_labels)
    chrf = chrf_metric.compute(predictions=all_preds, references=all_labels)

    model.train()  # Set back to training mode

    return {
        "bleu": bleu["score"],
        "rougeL": rouge["rougeL"],
        "chrf": chrf["score"],
        "sample_pred": all_preds[0] if all_preds else "",
        "sample_label": all_labels[0][0] if all_labels else "",
    }

___
# 6. Train

In [10]:
# --- SIMPLIFIED TRAINING LOOP FOR GERMAN GLOSS → VIETNAMESE ---
import time

best_bleu = float("-inf")
best_loss = float("inf")
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Training history
training_history = []

print(f"Starting training: {EPOCHS} epochs, {len(train_dataloader)} batches")

for epoch in range(EPOCHS):
    epoch_start = time.time()
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{EPOCHS}")

    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(device, non_blocking=True) for k, v in batch.items()}
        if MODEL_NAME.startswith("vinai"):
            batch.pop("token_type_ids", None)

        # Standard forward pass
        outputs = model(**batch)
        loss = outputs.loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()

        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        total_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS

        # Update progress bar with current average loss
        current_avg_loss = total_loss / (step + 1)
        progress_bar.set_postfix({"avg_loss": f"{current_avg_loss:.4f}"})

    # Epoch summary
    epoch_time = time.time() - epoch_start
    avg_loss = total_loss / len(train_dataloader)

    # Evaluate model
    metrics = evaluate_model(model, eval_dataloader)
    print(
        f"🎯Epoch {epoch + 1} - Metrics: BLEU={metrics['bleu']:.2f}, ROUGE-L={metrics['rougeL']:.2f}, chrF={metrics['chrf']:.2f}"
    )

    # Save training history
    training_history.append(
        {
            "epoch": epoch + 1,
            "train_loss": avg_loss,
            "bleu": metrics["bleu"],
            "rouge": metrics["rougeL"],
            "chrf": metrics["chrf"],
            "time": epoch_time,
        }
    )

    # Save best model
    if metrics["bleu"] > best_bleu:
        best_bleu = metrics["bleu"]
        print(f"New best BLEU: {best_bleu:.2f} - Model saved")

        model.save_pretrained(OUTPUT_DIR)
        tokenizer.save_pretrained(OUTPUT_DIR)

        import json

        with open(f"{OUTPUT_DIR}/training_history.json", "w") as f:
            json.dump(training_history, f, indent=2)

    torch.cuda.empty_cache()

print(f"Training completed! Best BLEU: {best_bleu:.2f}")
print(f"Model saved to: {OUTPUT_DIR}")

Starting training: 30 epochs, 589 batches


Epoch 1/30:   0%|          | 0/589 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

🎯Epoch 1 - Metrics: BLEU=4.86, ROUGE-L=0.26, chrF=13.90
New best BLEU: 4.86 - Model saved


Epoch 2/30:   0%|          | 0/589 [00:00<?, ?it/s]

...

Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

🎯Epoch 29 - Metrics: BLEU=24.21, ROUGE-L=0.53, chrF=39.72


Epoch 30/30:   0%|          | 0/589 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/22 [00:00<?, ?it/s]

🎯Epoch 30 - Metrics: BLEU=24.26, ROUGE-L=0.53, chrF=39.75
Training completed! Best BLEU: 24.60
Model saved to: ./gloss2vn_model


___
## 7. Testing

In [11]:
# Load the best model from training
from peft import PeftModel

best_model_path = OUTPUT_DIR
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME, use_safetensors=True)
final_model = PeftModel.from_pretrained(base_model, best_model_path).to(device)
final_tokenizer = AutoTokenizer.from_pretrained(best_model_path)

print("Evaluating best model on test set...")
test_metrics = evaluate_model(final_model, test_dataloader)
print(f"Test Results - BLEU: {test_metrics['bleu']:.3f}, ROUGE-L: {test_metrics['rougeL']:.3f}, chrF: {test_metrics['chrf']:.3f}")
print(f"Model saved to: {best_model_path}")
# !tree -h {best_model_path}
!ls

Evaluating best model on test set...


Evaluating:   0%|          | 0/27 [00:00<?, ?it/s]

Test Results - BLEU: 23.606, ROUGE-L: 0.515, chrF: 38.502
Model saved to: ./gloss2vn_model
gloss2vn_model	    PHOENIX-2014-T.dev.csv   PHOENIX-2014-T.train.csv
__notebook__.ipynb  PHOENIX-2014-T.test.csv


___
___
___